In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import torchaudio
import librosa
import os
from transformers import (
    Wav2Vec2Processor, 
    Wav2Vec2Model, 
    BertModel, 
    BertTokenizer
)
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Custom Dataset
import torch.nn.functional as F

class MultimodalEmotionDataset(Dataset):
    def __init__(self, csv_path, processor, tokenizer, max_length=128, target_audio_length=16000):
        # Read the CSV
        self.data = pd.read_csv(csv_path, sep='\t')
        
        # Add random text column if not exists
        if 'text' not in self.data.columns:
            print([f"Random text for {name}" for name in self.data['name']])
            self.data['text'] = [f"Random text for {name}" for name in self.data['name']]
        
        self.processor = processor
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.target_audio_length = target_audio_length
        
        # Mapping emotions to indices
        self.emotion_to_idx = {emotion: idx for idx, emotion in enumerate(self.data['emotion'].unique())}
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        
        # Load audio file
        speech_array, sampling_rate = torchaudio.load(row['path'])  # (Channels, Samples)
        
        # Convert to mono (if stereo, take the first channel)
        if speech_array.shape[0] > 1:
            speech_array = torch.mean(speech_array, dim=0, keepdim=True)
        
        speech_array = speech_array.squeeze().numpy()  # Convert to numpy array
        
        # Resample to target sampling rate
        speech_array = librosa.resample(y=speech_array, orig_sr=sampling_rate, target_sr=self.processor.feature_extractor.sampling_rate)
        
        # Ensure fixed length
        if len(speech_array) > self.target_audio_length:
            speech_array = speech_array[:self.target_audio_length]
        elif len(speech_array) < self.target_audio_length:
            padding = self.target_audio_length - len(speech_array)
            speech_array = np.pad(speech_array, (0, padding), mode='constant', constant_values=0)
    
        # Process audio with Wav2Vec2
        audio_inputs = self.processor(
            speech_array, 
            sampling_rate=self.processor.feature_extractor.sampling_rate, 
            return_tensors="pt"
        )
        
        # Process text
        text_inputs = self.tokenizer(
            row['text'], 
            max_length=self.max_length, 
            padding='max_length', 
            truncation=True, 
            return_tensors="pt"
        )
        
        # Get emotion label
        label = self.emotion_to_idx[row['emotion']]
        
        return {
            'audio_input': audio_inputs.input_values.squeeze(),  # Ensure correct shape
            'audio_mask': audio_inputs.attention_mask.squeeze(),
            'text_input_ids': text_inputs['input_ids'].squeeze(),
            'text_attention_mask': text_inputs['attention_mask'].squeeze(),
            'label': label
        }


    # def __getitem__(self, idx):
    #     row = self.data.iloc[idx]
        
    #     # Process Audio
    #     speech_array, sampling_rate = torchaudio.load(row['path'])
    #     speech_array = speech_array.squeeze().numpy()
    #     # speech_array = librosa.resample(np.asarray(speech_array), sampling_rate, self.processor.feature_extractor.sampling_rate)
    #     speech_array = librosa.resample(y=np.asarray(speech_array), orig_sr=sampling_rate, target_sr=self.processor.feature_extractor.sampling_rate)
    #     # Pad or truncate audio
    #     if len(speech_array) > self.target_audio_length:
    #         speech_array = speech_array[:self.target_audio_length]
    #     elif len(speech_array) < self.target_audio_length:
    #         padding = self.target_audio_length - len(speech_array)
    #         speech_array = np.pad(speech_array, (0, padding), mode='constant', constant_values=0)
        
    #     audio_inputs = self.processor(
    #         speech_array, 
    #         sampling_rate=self.processor.feature_extractor.sampling_rate, 
    #         return_tensors="pt"
    #     )
        
    #     # Process Text
    #     text_inputs = self.tokenizer(
    #         row['text'], 
    #         max_length=self.max_length, 
    #         padding='max_length', 
    #         truncation=True, 
    #         return_tensors="pt"
    #     )
        
    #     # Get emotion label
    #     label = self.emotion_to_idx[row['emotion']]
        
    #     return {
    #         'audio_input': audio_inputs.input_values.squeeze(),
    #         'audio_mask': audio_inputs.attention_mask.squeeze(),
    #         'text_input_ids': text_inputs['input_ids'].squeeze(),
    #         'text_attention_mask': text_inputs['attention_mask'].squeeze(),
    #         'label': label
    #     }


class MultimodalEmotionClassifier(nn.Module):
    def __init__(self, num_labels, audio_model_path, text_model_path):
        super().__init__()
        
        # Audio Encoder (Wav2Vec2)
        self.audio_encoder = Wav2Vec2Model.from_pretrained(audio_model_path)
        
        # Text Encoder (BERT)
        self.text_encoder = BertModel.from_pretrained(text_model_path)
        
        # Freeze pretrained encoders (optional)
        for param in self.audio_encoder.parameters():
            param.requires_grad = False
        for param in self.text_encoder.parameters():
            param.requires_grad = False
        
        # Feature dimensions
        self.audio_feature_dim = self.audio_encoder.config.hidden_size
        self.text_feature_dim = self.text_encoder.config.hidden_size
        
        # Projection layers to common dimension
        common_dim = 512
        self.audio_projection = nn.Linear(self.audio_feature_dim, common_dim)
        self.text_projection = nn.Linear(self.text_feature_dim, common_dim)
        
        # Attention mechanism
        self.attention = nn.Linear(common_dim * 2, 2)
        
        # Fusion Layer
        self.fusion_layers = nn.Sequential(
            nn.Linear(common_dim * 2, common_dim),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(common_dim, num_labels)
        )
        
        self.audio_norm = nn.LayerNorm(self.audio_feature_dim)
        self.text_norm = nn.LayerNorm(self.text_feature_dim)

    def forward(self, audio_input, audio_mask, text_input_ids, text_attention_mask):
        # Extract audio features
        audio_outputs = self.audio_encoder(audio_input, attention_mask=audio_mask)
        audio_features = torch.mean(audio_outputs.last_hidden_state, dim=1)
        audio_features = self.audio_norm(audio_features)
        
        # Extract text features
        text_outputs = self.text_encoder(text_input_ids, attention_mask=text_attention_mask)
        text_features = text_outputs.pooler_output
        text_features = self.text_norm(text_features)
        
        # Project to common dimension
        audio_projected = self.audio_projection(audio_features)
        text_projected = self.text_projection(text_features)
        
        # Compute attention weights
        combined = torch.cat([audio_projected, text_projected], dim=1)
        # # attention_weights = F.softmax(self.attention(combined), dim=1)
        # bias = torch.tensor([2.0, 1.0], device=combined.device)  # Give more weight to audio
        # attention_logits = self.attention(combined) + bias
        # attention_weights = F.softmax(attention_logits, dim=1)

        
        # # # Apply attention weights
        # modal_features = torch.cat([
        #     audio_projected * attention_weights[:, 0].unsqueeze(1).expand_as(audio_projected),
        #     text_projected * attention_weights[:, 1].unsqueeze(1).expand_as(text_projected)
        # ], dim=1)
        # Updated:
        bias = torch.tensor([2.0, 1.0], device=combined.device)  # More weight to audio
        attention_logits = self.attention(torch.cat([audio_projected, text_projected], dim=1)) + bias
        attention_weights = F.softmax(attention_logits, dim=1)
        
        modal_features = torch.cat([
            audio_projected * attention_weights[:, 0].unsqueeze(1).expand_as(audio_projected),
            text_projected * attention_weights[:, 1].unsqueeze(1).expand_as(text_projected)
        ], dim=1)
        # modal_features = torch.cat([
        #     0.7 * audio_projected,
        #     0.3 * text_projected
        # ], dim=1)
        # Classification
        logits = self.fusion_layers(modal_features)
        
        return logits, attention_weights

    # def forward(self, audio_input, audio_mask, text_input_ids, text_attention_mask):
    #     # Extract audio features
    #     audio_outputs = self.audio_encoder(
    #         audio_input, 
    #         attention_mask=audio_mask
    #     )
    #     audio_features = torch.mean(audio_outputs.last_hidden_state, dim=1)
        
    #     # Extract text features
    #     text_outputs = self.text_encoder(
    #         text_input_ids, 
    #         attention_mask=text_attention_mask
    #     )
    #     text_features = text_outputs.pooler_output
        
    #     # Concatenate features
    #     combined_features = torch.cat([audio_features, text_features], dim=1)
        
    #     # Classification
    #     logits = self.fusion_layers(combined_features)
        
    #     return logits

def train_model(model, train_loader, val_loader, device, epochs=10, learning_rate=1e-5, text_penalty=0.1):
    # Loss and Optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Create models directory if it doesn't exist
    os.makedirs('saved_models', exist_ok=True)
    
    best_val_accuracy = 0.0
    
    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_losses = []
        train_preds = []
        train_true = []
        
        for batch in train_loader:
            # Move data to device
            audio_input = batch['audio_input'].to(device)
            audio_mask = batch['audio_mask'].to(device)
            text_input_ids = batch['text_input_ids'].to(device)
            text_attention_mask = batch['text_attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs_tuple = model(
                audio_input, 
                audio_mask, 
                text_input_ids, 
                text_attention_mask
            )
            
            # Handle the tuple return value
            if isinstance(outputs_tuple, tuple):
                outputs, attention_weights = outputs_tuple
                
                # Compute classification loss
                class_loss = criterion(outputs, labels)
                
                # Add penalty for relying too much on text features
                text_reliance_penalty = text_penalty * attention_weights[:, 1].mean()
                
                # Total loss
                loss = class_loss + text_reliance_penalty
            else:
                # For backward compatibility with original model
                outputs = outputs_tuple
                loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            # Track training metrics
            train_losses.append(loss.item())
            preds = torch.argmax(outputs, dim=1)
            train_preds.extend(preds.cpu().numpy())
            train_true.extend(labels.cpu().numpy())
        
        # Validation Phase
        model.eval()
        val_losses = []
        val_preds = []
        val_true = []
        
        with torch.no_grad():
            for batch in val_loader:
                # Move data to device
                audio_input = batch['audio_input'].to(device)
                audio_mask = batch['audio_mask'].to(device)
                text_input_ids = batch['text_input_ids'].to(device)
                text_attention_mask = batch['text_attention_mask'].to(device)
                labels = batch['label'].to(device)
                
                # Forward pass
                outputs_tuple = model(
                    audio_input, 
                    audio_mask, 
                    text_input_ids, 
                    text_attention_mask
                )
                
                # Handle the tuple return value
                if isinstance(outputs_tuple, tuple):
                    outputs, _ = outputs_tuple
                    loss = criterion(outputs, labels)
                else:
                    outputs = outputs_tuple
                    loss = criterion(outputs, labels)
                
                val_losses.append(loss.item())
                
                # Track validation metrics
                preds = torch.argmax(outputs, dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_true.extend(labels.cpu().numpy())
        
        # Compute metrics
        train_accuracy = accuracy_score(train_true, train_preds)
        val_accuracy = accuracy_score(val_true, val_preds)
        
        # Print epoch summary
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Train Loss: {np.mean(train_losses):.4f}, Train Accuracy: {train_accuracy:.4f}")
        print(f"Val Loss: {np.mean(val_losses):.4f}, Val Accuracy: {val_accuracy:.4f}")
        
        # Save best model
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_val_accuracy': best_val_accuracy,
                'epoch': epoch
            }, f'saved_models/best_multimodal_model.pth')
            
            print(f"Saved new best model with validation accuracy: {best_val_accuracy:.4f}")
        
        # Optional: Print classification report for validation set
        if epoch % 5 == 0:
            print("\nValidation Classification Report:")
            print(classification_report(val_true, val_preds))
    
    return model

def main():
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Paths and Configurations
    audio_model_path = "./emotion_recognition_model"
    text_model_path = "bert-base-uncased"
    
    # Initialize Processors
    audio_processor = Wav2Vec2Processor.from_pretrained(audio_model_path)
    text_tokenizer = BertTokenizer.from_pretrained(text_model_path)
    
    # Create Datasets
    train_dataset = MultimodalEmotionDataset("dataset/train.csv", audio_processor, text_tokenizer)
    test_dataset = MultimodalEmotionDataset("dataset/test.csv", audio_processor, text_tokenizer)
    
    # Split train into train and validation
    train_dataset, val_dataset = torch.utils.data.random_split(
        train_dataset, 
        [int(len(train_dataset)*0.8), len(train_dataset)-int(len(train_dataset)*0.8)]
    )
    
    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
    
    # Get number of labels
    num_labels = len(train_dataset.dataset.emotion_to_idx)
    
    # Initialize Model
    model = MultimodalEmotionClassifier(
        num_labels=num_labels, 
        audio_model_path=audio_model_path, 
        text_model_path=text_model_path
    ).to(device)
    print(model)
    print(f"Model Initialized with {num_labels} emotion classes")
    print("Emotion to Index mapping:", train_dataset.dataset.emotion_to_idx)
    
    # Train Model
    trained_model = train_model(model, train_loader, val_loader, device, epochs=50)
    
    # Optional: Load and evaluate best saved model
    best_model_path = 'saved_models/best_multimodal_model.pth'
    checkpoint = torch.load(best_model_path)
    
    # Reinitialize model and load state dict
    best_model = MultimodalEmotionClassifier(
        num_labels=num_labels, 
        audio_model_path=audio_model_path, 
        text_model_path=text_model_path
    ).to(device)
    best_model.load_state_dict(checkpoint['model_state_dict'])
    
    # Test model
    # Test model
    best_model.eval()
    test_preds = []
    test_true = []
    
    with torch.no_grad():
        for batch in test_loader:
            # Move data to device
            audio_input = batch['audio_input'].to(device)
            audio_mask = batch['audio_mask'].to(device)
            text_input_ids = batch['text_input_ids'].to(device)
            text_attention_mask = batch['text_attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            # Forward pass
            outputs_tuple = best_model(
                audio_input, 
                audio_mask, 
                text_input_ids, 
                text_attention_mask
            )
            
            # Handle the tuple return value
            if isinstance(outputs_tuple, tuple):
                outputs, _ = outputs_tuple
            else:
                outputs = outputs_tuple
            
            # Track test metrics
            preds = torch.argmax(outputs, dim=1)
            test_preds.extend(preds.cpu().numpy())
            test_true.extend(labels.cpu().numpy())
    # Print test classification report
    print("\nTest Classification Report:")
    print(classification_report(test_true, test_preds, 
        target_names=list(train_dataset.dataset.emotion_to_idx.keys())))
    
if __name__ == "__main__":
    main()

MultimodalEmotionClassifier(
  (audio_encoder): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=T

C:\Python\py311env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python\py311env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python\py311env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 2/50
Train Loss: 1.6136, Train Accuracy: 0.2329
Val Loss: 1.6055, Val Accuracy: 0.2468
Saved new best model with validation accuracy: 0.2468
Epoch 3/50
Train Loss: 1.6113, Train Accuracy: 0.2410
Val Loss: 1.5987, Val Accuracy: 0.2532
Saved new best model with validation accuracy: 0.2532
Epoch 4/50
Train Loss: 1.5902, Train Accuracy: 0.2557
Val Loss: 1.5977, Val Accuracy: 0.2468
Epoch 5/50
Train Loss: 1.6038, Train Accuracy: 0.2231
Val Loss: 1.5949, Val Accuracy: 0.2662
Saved new best model with validation accuracy: 0.2662
Epoch 6/50
Train Loss: 1.6038, Train Accuracy: 0.2378
Val Loss: 1.5934, Val Accuracy: 0.2597

Validation Classification Report:
              precision    recall  f1-score   support

           0       0.40      0.05      0.09        38
           1       0.24      0.41      0.31        27
           2       0.21      0.46      0.29        26
           3       0.38      0.14      0.21        35
           4       0.30      0.36      0.33        28

    accuracy

C:\Python\py311env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python\py311env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python\py311env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 17/50
Train Loss: 1.5755, Train Accuracy: 0.2785
Val Loss: 1.5695, Val Accuracy: 0.2922
Epoch 18/50
Train Loss: 1.5880, Train Accuracy: 0.2818
Val Loss: 1.5705, Val Accuracy: 0.2727
Epoch 19/50
Train Loss: 1.5879, Train Accuracy: 0.2443
Val Loss: 1.5701, Val Accuracy: 0.2727
Epoch 20/50
Train Loss: 1.5779, Train Accuracy: 0.2687
Val Loss: 1.5671, Val Accuracy: 0.2857
Epoch 21/50
Train Loss: 1.5779, Train Accuracy: 0.2964
Val Loss: 1.5651, Val Accuracy: 0.2857

Validation Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.27      0.52      0.36        27
           2       0.19      0.35      0.25        26
           3       0.42      0.37      0.39        35
           4       0.32      0.29      0.30        28

    accuracy                           0.29       154
   macro avg       0.24      0.30      0.26       154
weighted avg       0.23      0.29      0.25       154



C:\Python\py311env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python\py311env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python\py311env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 22/50
Train Loss: 1.5716, Train Accuracy: 0.2866
Val Loss: 1.5601, Val Accuracy: 0.2987
Epoch 23/50
Train Loss: 1.5775, Train Accuracy: 0.2704
Val Loss: 1.5569, Val Accuracy: 0.3117
Saved new best model with validation accuracy: 0.3117
Epoch 24/50
Train Loss: 1.5712, Train Accuracy: 0.2834
Val Loss: 1.5580, Val Accuracy: 0.3052
Epoch 25/50
Train Loss: 1.5820, Train Accuracy: 0.2818
Val Loss: 1.5588, Val Accuracy: 0.2792
Epoch 26/50
Train Loss: 1.5743, Train Accuracy: 0.2850
Val Loss: 1.5567, Val Accuracy: 0.2922

Validation Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.29      0.44      0.35        27
           2       0.18      0.35      0.24        26
           3       0.44      0.43      0.43        35
           4       0.31      0.32      0.32        28

    accuracy                           0.29       154
   macro avg       0.25      0.31      0.27       154
weighte